In [0]:
import datetime
import requests
import json # I find out that I need this to read the "return" from the weather API
from pyspark.sql.functions import (
    col,            
    lit,            
    when,           
    to_timestamp,   
    date_format,    
    hour,           
    dayofweek,      
    count,          
    sum             
)
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
    TimestampType
)

print("Toolbox opened successfully 🧙‍♂️")

Toolbox opened successfully 🧙‍♂️


In [0]:
from pyspark.sql.functions import col, hour, count
df_silver = spark.read.table("personal_projects.gru_airport.silver_vra")
# What is the most common hour of the day for flights from GRU airport?
df_flux_hour = (
    df_silver
    # only dep. from GRU
    .filter(col("sigla_icao_origem") == "SBGR")
    #also here after the 1st run I had to create a new filter for the canceled flights
    .filter(col("partida_real").isNotNull())
    # creat a new column with the hour rounded
    .withColumn("hora_dia", hour(col("partida_real")))
    # group and count
    .groupBy("hora_dia")
    .agg(count("*").alias("total_voos"))
    # order to read by the highest
    .orderBy(col("total_voos").desc())
)

display(df_flux_hour)

hora_dia,total_voos
23,1017
9,1003
22,986
7,919
8,904
17,792
21,692
15,645
12,580
13,579


Databricks visualization. Run in Databricks to view.

In [0]:
(df_flux_hour.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable("personal_projects.gru_airport.gold_flux_schedule"))

print("Gold table saved successfully - Scheduled hour")

Gold table saved successfully - Scheduled hour


In [0]:
# Airlines Ranking by flights

df_ranking_airlines = (
    df_silver
    .filter(col("sigla_icao_origem") == "SBGR")
    .filter(col("partida_real").isNotNull())
    #groupby airlines
    .groupBy("empresa_aerea")
    .agg(count("*").alias("total_voos"))
    
    .orderBy(col("total_voos").desc())
)

display(df_ranking_airlines)

empresa_aerea,total_voos
TAM LINHAS AÉREAS S.A.,5804
GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.),2962
AZUL LINHAS AÉREAS BRASILEIRAS S/A,846
LATAM AIRLINES GROUP (EX - LAN AIRLINES S/A),197
COMPAÑIA PANAMEÑA DE AVIACION S.A. (COPA AIRLINES),180
"AMERICAN AIRLINES, INC.",154
AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA,137
"UNITED AIRLINES, INC",123
PASSAREDO TRANSPORTES AÉREOS S.A.,115
SIDERAL LINHAS AÉREAS LTDA.,110


Databricks visualization. Run in Databricks to view.

In [0]:
# second table - airline ranking

(df_ranking_airlines.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable("personal_projects.gru_airport.gold_ranking_airlines"))

print("Ranking Airlines Table saved successfully")

Ranking Airlines Table saved successfully


In [0]:
%sql
-- cancelled flights - ranking of the airlines:

CREATE OR REPLACE TABLE personal_projects.gru_airport.gold_cancelamentos AS
SELECT 
    empresa_aerea,
    COUNT(*) as total_cancelamentos
FROM 
    personal_projects.gru_airport.silver_vra
WHERE 
    situacao_voo = 'CANCELADO'
GROUP BY 
    empresa_aerea
ORDER BY 
    total_cancelamentos DESC;
SELECT * FROM personal_projects.gru_airport.gold_cancelamentos;

empresa_aerea,total_cancelamentos
TAM LINHAS AÉREAS S.A.,169
ETHIOPIAN AIRLINES GROUP,151
SIDERAL LINHAS AÉREAS LTDA.,81
AZUL LINHAS AÉREAS BRASILEIRAS S/A,68
GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.),55
SKY AIRLINE S.A.,40
ATLAS AIR INC,36
COMPAGNIE NATIONALE ROYAL AIR MAROC,26
LATAM AIRLINES GROUP (EX - LAN AIRLINES S/A),20
ITALIA TRANSPORTO AEREO S.P.A.,17


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- average delay::
CREATE OR REPLACE TABLE personal_projects.gru_airport.gold_atrasos AS
SELECT 
    empresa_aerea,
    -- calculate the average delay in minutes
    ROUND(AVG((unix_timestamp(partida_real) - unix_timestamp(partida_prevista)) / 60), 2) as media_atraso_minutos,
    COUNT(*) as total_voos_considerados
FROM 
    personal_projects.gru_airport.silver_vra
WHERE 
    partida_real IS NOT NULL
GROUP BY 
    empresa_aerea
-- we are excluding arlines that has only a few flights (1 or 2)
HAVING 
    total_voos_considerados > 10
ORDER BY 
    media_atraso_minutos DESC;
SELECT * FROM personal_projects.gru_airport.gold_atrasos;

empresa_aerea,media_atraso_minutos,total_voos_considerados
ATLAS AIR INC,179.4,56
LAN CARGO S.A.,80.24,29
"AMERICAN AIRLINES, INC.",68.62,309
SOUTH AFRICAN AIRWAYS STATE OWNED COMPANY (SOC) LIMITED,49.49,35
BOA BOLIVIANA DE AVIACIÓN,47.6,124
AIR EUROPA LINEAS AEREAS SOCIEDAD ANONIMA,45.16,64
AIR CHINA,43.69,26
SOCIÉTÉ AIR FRANCE,41.34,124
ABSA - AEROLINHAS BRASILEIRAS S.A.,41.21,71
DELTA AIR LINES INC.,39.32,211


Databricks visualization. Run in Databricks to view.